# Temporal Difference Learning

The code in this notebook implements different methods for simulating and analyzing a **Random Walk** environment using **Temporal Difference (TD)** learning algorithms.

### Purpose of the Code:
1. **Random Walk Environment**: The simulation represents a random walk between two terminal states (0 and 6). The agent starts in the middle state (3) and takes random actions (either left or right). The goal is to reach the terminal state on the right (6), which provides a reward of 1, while the left terminal state (0) gives no reward.

2. **Temporal Difference (TD) Learning**: 
   - The notebook employs **TD(0)**, a reinforcement learning algorithm, to estimate the state-value function $ V(s) $, which represents the expected reward when starting from a given state.
   - It explores how well the algorithm estimates the true value function through several episodes, updating the value of each state based on the agent's experience during the episodes.

3. **Policy Comparison**: The notebook also includes two different policies for action selection, although it mainly focuses on how the value function is learned by interacting with the environment. 

4. **Evaluation of Learning**: The success of the learning process is evaluated by comparing the estimated value function with a "true" value function. The performance is measured using Root Mean Square (RMS) error to quantify how close the learned values are to the true values.

### Sections of the Code:
1. **Defining the Random Walk Environment**: 
   The environment has 7 states, where the agent performs random actions to move either left or right. The agent’s goal is to reach the rewarding state (state 6).

2. **TD(0) Algorithm**: 
   The core of the code implements TD(0), where state values are updated incrementally after each action based on the reward and the value of the next state.

3. **Evaluation**: 
   After running a set number of episodes, the learned value function is printed to observe how well the agent has learned to predict the expected rewards for each state.

In [1]:
import numpy as np
from tqdm import tqdm
import random
np.set_printoptions(precision=4, suppress=True)
random.seed(42)

In [2]:
class RandomWalkEnv:
    def __init__(self):
        self.states = list(range(7))  # States from 0 to 6
        self.terminal_states = [0, 6]
        self.current_state = 3  # Initial position is "C"
    
    def reset(self):
        self.current_state = 3  # Reset to initial state
        return self.current_state

    def step(self, action):
        # If already in a terminal state, return immediately
        if self.current_state in self.terminal_states:
            return self.current_state, 0, True
        
        # Move to the next state based on the action
        # if action == 0:  # Move left
        #     next_state = self.current_state - 1
        # else:  # Move right
        #     next_state = self.current_state + 1
        next_state = self.current_state + (1 if action == 1 else -1)
        
        # Ensure next_state is within bounds
        next_state = max(0, min(next_state, 6))
        
        # Determine reward and done status
        reward = 1 if next_state == 6 else 0
        done = next_state in self.terminal_states
        
        # Update the current state
        self.current_state = next_state
        
        return self.current_state, reward, done

In [3]:
def run_simulation(env, policy, num_steps=10):
    state = env.reset()
    print("Starting from State 3...")
    print(f"{'Action':<10} {'State':<10} {'Reward':<10} {'Done':<10}")
    print("-" * 40)
    
    for _ in range(num_steps):
        action = np.random.choice([0, 1], p=policy[state])
        state, reward, done = env.step(action)
        print(f"{ACTION_NAMES[action]:<10} {state:<10} {reward:<10} {done:<10}")
        
        if done:
            state = env.reset()  # Reset the environment if done
            print("Restarting from State 3...")
    
    return state, reward, done

In [4]:
# Testing the environment
ACTION_NAMES = ["LEFT", "RIGHT"]
env_rw = RandomWalkEnv()
policy_b = np.array([[0.4, 0.6], [0.45, 0.55], [0.5, 0.5], [0.7, 0.3], [0.5, 0.5], [0.6, 0.4], [0.9, 0.1]])
policy_pi = np.array([[0.5, 0.5]] * 7) # uniform probability

In [5]:
state, reward, done = run_simulation(env_rw, policy_pi)

Starting from State 3...
Action     State      Reward     Done      
----------------------------------------
LEFT       2          0          0         
RIGHT      3          0          0         
RIGHT      4          0          0         
LEFT       3          0          0         
LEFT       2          0          0         
LEFT       1          0          0         
LEFT       0          0          1         
Restarting from State 3...
LEFT       2          0          0         
LEFT       1          0          0         
LEFT       0          0          1         
Restarting from State 3...


In [6]:
state, reward, done = run_simulation(env_rw, policy_b)

Starting from State 3...
Action     State      Reward     Done      
----------------------------------------
LEFT       2          0          0         
RIGHT      3          0          0         
LEFT       2          0          0         
LEFT       1          0          0         
LEFT       0          0          1         
Restarting from State 3...
LEFT       2          0          0         
RIGHT      3          0          0         
LEFT       2          0          0         
LEFT       1          0          0         
LEFT       0          0          1         
Restarting from State 3...


In [7]:
def run_episode_with_timestep(env, policy):
    state = env.reset()
    episode = []
    time_step = 0
    
    print("Starting from State 3...")
    print(f"{'Time Step':<10} {'Action':<10} {'State':<10} {'Reward':<10} {'Done':<10}")
    print("-" * 50)
    
    while True:
        action = np.random.choice([0, 1], p=policy[state])
        next_state, reward, done = env.step(action)
        episode.append((time_step, state, action, reward, next_state, done))
        
        print(f"{time_step:<10} {ACTION_NAMES[action]:<10} {next_state:<10} {reward:<10} {done:<10}")
        
        if done:
            break
        
        state = next_state
        time_step += 1
        
    return episode

In [8]:
episode_policy_pi = run_episode_with_timestep(env_rw, policy_pi)

Starting from State 3...
Time Step  Action     State      Reward     Done      
--------------------------------------------------
0          RIGHT      4          0          0         
1          LEFT       3          0          0         
2          LEFT       2          0          0         
3          LEFT       1          0          0         
4          LEFT       0          0          1         


In [9]:
# episode_policy_pi

In [10]:
episode_policy_b = run_episode_with_timestep(env_rw, policy_b)

Starting from State 3...
Time Step  Action     State      Reward     Done      
--------------------------------------------------
0          LEFT       2          0          0         
1          LEFT       1          0          0         
2          RIGHT      2          0          0         
3          RIGHT      3          0          0         
4          LEFT       2          0          0         
5          RIGHT      3          0          0         
6          LEFT       2          0          0         
7          LEFT       1          0          0         
8          RIGHT      2          0          0         
9          RIGHT      3          0          0         
10         LEFT       2          0          0         
11         RIGHT      3          0          0         
12         LEFT       2          0          0         
13         RIGHT      3          0          0         
14         LEFT       2          0          0         
15         RIGHT      3          0          

In [11]:
# episode_policy_b

In [12]:
def run_multiple_episodes(env, policy_pi, num_episodes=10):
    successes = 0
    
    for episode_num in range(num_episodes):
        state = env.reset()
        time_step = 0
        success = False
        
        print(f"Starting Episode {episode_num + 1} from State 3...")
        print(f"{'Time Step':<10} {'Action':<10} {'State':<10} {'Reward':<10} {'Done':<10}")
        print("-" * 50)
        
        while True:
            action = np.random.choice([0, 1], p=policy_pi[state])
            next_state, reward, done = env.step(action)
            time_step += 1
            
            print(f"{time_step:<10} {ACTION_NAMES[action]:<10} {next_state:<10} {reward:<10} {done:<10}")
            
            if done or next_state == 6:
                success = next_state == 6
                break
            
            state = next_state
        
        if success:
            print("Successfully reached State 6!")
            successes += 1
        else:
            print("Failed to reach State 6.")
        
        print("=" * 50)
    
    success_rate = successes / num_episodes
    print(f"Success Rate: {success_rate * 100:.2f}%")
    
    return success_rate

In [13]:
success_rate_policy_pi = run_multiple_episodes(env_rw, policy_pi)

Starting Episode 1 from State 3...
Time Step  Action     State      Reward     Done      
--------------------------------------------------
1          RIGHT      4          0          0         
2          RIGHT      5          0          0         
3          RIGHT      6          1          1         
Successfully reached State 6!
Starting Episode 2 from State 3...
Time Step  Action     State      Reward     Done      
--------------------------------------------------
1          RIGHT      4          0          0         
2          RIGHT      5          0          0         
3          RIGHT      6          1          1         
Successfully reached State 6!
Starting Episode 3 from State 3...
Time Step  Action     State      Reward     Done      
--------------------------------------------------
1          RIGHT      4          0          0         
2          LEFT       3          0          0         
3          LEFT       2          0          0         
4          LEFT       

In [14]:
success_rate_policy_b = run_multiple_episodes(env_rw, policy_b)

Starting Episode 1 from State 3...
Time Step  Action     State      Reward     Done      
--------------------------------------------------
1          RIGHT      4          0          0         
2          LEFT       3          0          0         
3          LEFT       2          0          0         
4          LEFT       1          0          0         
5          LEFT       0          0          1         
Failed to reach State 6.
Starting Episode 2 from State 3...
Time Step  Action     State      Reward     Done      
--------------------------------------------------
1          LEFT       2          0          0         
2          LEFT       1          0          0         
3          LEFT       0          0          1         
Failed to reach State 6.
Starting Episode 3 from State 3...
Time Step  Action     State      Reward     Done      
--------------------------------------------------
1          RIGHT      4          0          0         
2          RIGHT      5         

In [15]:
def td_zero(env, policy, alpha=0.01, gamma=1.0, episodes=8000):
    V = np.zeros(7)  # Value function initialized to zero for all states    
    for episode in tqdm(range(episodes), desc="Episodes"):
        state = env.reset()
        done = False
        # episode_reward = 0
        while not done:
            action = np.random.choice([0, 1], p=policy[state])
            next_state, reward, done = env.step(action)
            V[state] += alpha * (reward + gamma * V[next_state] - V[state])
            state = next_state
            # episode_reward += reward 
        # if episode % 10 == 0:
        #     print(f"Episode:{episode}, Reward:{episode_reward}")
    return V

In [16]:
VF_true = np.arange(1,6)/6
VF_true

array([0.1667, 0.3333, 0.5   , 0.6667, 0.8333])

In [17]:
def evaluate_policy(env, policy, V_true, value_function_estimator, num_runs=10):
    rms_errors = []
    
    for _ in range(num_runs):
        # Use the provided value function estimator
        value_function = value_function_estimator(env, policy)
        print("Estimated Value Function (excluding boundaries):", value_function[1:-1])
        
        # Compute RMS error
        rms_error = np.sqrt(np.mean((value_function[1:-1] - V_true) ** 2))
        print(f"RMS Error: {rms_error}")
        
        rms_errors.append(rms_error)
    
    return rms_errors


In [18]:
rms_errors_policy_pi = evaluate_policy(env_rw, policy_pi, VF_true, td_zero)

Episodes: 100%|███████████████████████████████████████████████████████████████████| 8000/8000 [00:08<00:00, 956.52it/s]


Estimated Value Function (excluding boundaries): [0.1494 0.3049 0.4655 0.6481 0.8102]
RMS Error: 0.025220523424311532


Episodes: 100%|███████████████████████████████████████████████████████████████████| 8000/8000 [00:08<00:00, 991.55it/s]


Estimated Value Function (excluding boundaries): [0.1553 0.2892 0.4564 0.6333 0.8138]
RMS Error: 0.03307417916416644


Episodes: 100%|██████████████████████████████████████████████████████████████████| 8000/8000 [00:07<00:00, 1001.64it/s]


...

Estimated Value Function (excluding boundaries): [0.1641 0.3509 0.5275 0.6876 0.8356]
RMS Error: 0.01738861827080628


Episodes: 100%|███████████████████████████████████████████████████████████████████| 8000/8000 [00:08<00:00, 956.88it/s]


Estimated Value Function (excluding boundaries): [0.174  0.3547 0.5219 0.7039 0.8561]
RMS Error: 0.024059583489499825


Episodes: 100%|███████████████████████████████████████████████████████████████████| 8000/8000 [00:08<00:00, 985.47it/s]

Estimated Value Function (excluding boundaries): [0.1781 0.3489 0.5067 0.6683 0.838 ]
RMS Error: 0.009414182117313048


In [19]:
# Adaptive learning rate decay
def td_zero_adaptive(env, policy, alpha_init=0.15, alpha_min=0.01, decay_rate=0.9, gamma=1.0, episodes=2000):
    V = np.zeros(7)  # Value function initialized to zero for all states  
    alpha = alpha_init
    
    for episode in tqdm(range(episodes), desc="Episodes"):
        state = env.reset()
        done = False
        old_V_value = np.copy(V)
        
        while not done:
            action = np.random.choice([0, 1], p=policy[state])
            next_state, reward, done = env.step(action)
            V[state] += alpha * (reward + gamma * V[next_state] - V[state])
            state = next_state
        
        # Calculate average change in value function
        avg_V_change = np.mean(np.abs(V - old_V_value))
        
        # Update alpha if the change is below the threshold
        if avg_V_change < 0.05:
            alpha = max(alpha * decay_rate, alpha_min)
    
    return V

In [20]:
rms_errors_policy_pi = evaluate_policy(env_rw, policy_pi, VF_true, td_zero_adaptive)

Episodes: 100%|██████████████████████████████████████████████████████████████████| 2000/2000 [00:01<00:00, 1033.18it/s]


Estimated Value Function (excluding boundaries): [0.1493 0.3115 0.4798 0.6667 0.8535]
RMS Error: 0.017853915197372932


Episodes: 100%|███████████████████████████████████████████████████████████████████| 2000/2000 [00:02<00:00, 965.78it/s]


Estimated Value Function (excluding boundaries): [0.1634 0.3687 0.5368 0.6928 0.8484]
RMS Error: 0.02654510470078203


Episodes: 100%|███████████████████████████████████████████████████████████████████| 2000/2000 [00:02<00:00, 980.37it/s]


...

Estimated Value Function (excluding boundaries): [0.1349 0.3143 0.4931 0.649  0.8298]
RMS Error: 0.01867912696757387


Episodes: 100%|███████████████████████████████████████████████████████████████████| 2000/2000 [00:02<00:00, 902.22it/s]


Estimated Value Function (excluding boundaries): [0.1667 0.3557 0.525  0.7042 0.8391]
RMS Error: 0.022665135948945314


Episodes: 100%|███████████████████████████████████████████████████████████████████| 2000/2000 [00:02<00:00, 928.02it/s]

Estimated Value Function (excluding boundaries): [0.1322 0.2784 0.4553 0.644  0.8451]
RMS Error: 0.03702779840527947


### Exercise 6.7 
Design an off-policy version of the $TD(0)$ update that can be used with arbitrary target policy $\pi$ and covering behavior policy $b$, using at each step $t$ the importance sampling ratio $\rho_{t:t}$

1. **Importance Sampling Ratio**:
   The importance sampling ratio at step $ t $ is defined as:
   $$
   \rho_{t:t} = \frac{\pi(A_t | S_t)}{b(A_t | S_t)}
   $$
   This ratio compares the probability of the action taken under the target policy to the probability of the same action under the behavior policy.

2. **Off-Policy Update Rule**:
   The off-policy $ TD(0) $ update can then be defined as:
   $$
   V(S_t) \leftarrow V(S_t) + \alpha \rho_{t:t} \left( R_{t+1} + \gamma V(S_{t+1}) - V(S_t) \right)
   $$
   where:
   - $ \alpha $ is the step size (learning rate).
   - $ \gamma $ is the discount factor.
   - $ S_{t+1} $ is the state resulting from taking action $ A_t $ in state $ S_t $.

In [21]:
def off_policy_td_zero(env, beh_policy, alpha=0.01, gamma=1.0, episodes=2000):
    # Initialize value function to zero for all states
    V = np.zeros(len(env.states))
    pi_prob = 0.5  # Probability for target policy π (uniform)

    for episode in tqdm(range(episodes), desc="Episodes"):
        state = env.reset()
        done = False

        while not done:
            action = np.random.choice([0, 1], p=beh_policy[state])
            next_state, reward, done = env.step(action)

            if not done:
                rho = pi_prob / beh_policy[state, action]
                td_target = reward + gamma * V[next_state]
                V[state] += alpha * rho * (td_target - V[state])
                # Debugging output
                # print(f"Episode: {episode}, State: {state}, Action: {action}, Next State: {next_state}, Reward: {reward}, V: {V}")
            else:
                # Update the last state when reaching terminal state
                rho = pi_prob / beh_policy[state, action]
                V[state] += alpha * rho * (reward - V[state])  # Update value for the terminal state as well

            state = next_state

    return V

In [22]:
rms_errors_off_policy = evaluate_policy(env_rw, policy_b, VF_true, off_policy_td_zero)

Episodes: 100%|███████████████████████████████████████████████████████████████████| 2000/2000 [00:02<00:00, 903.12it/s]


Estimated Value Function (excluding boundaries): [0.1542 0.3038 0.4822 0.6662 0.8471]
RMS Error: 0.017494503321701674


Episodes: 100%|███████████████████████████████████████████████████████████████████| 2000/2000 [00:02<00:00, 830.05it/s]


Estimated Value Function (excluding boundaries): [0.1712 0.3189 0.5037 0.6654 0.837 ]
RMS Error: 0.007191204199005797


Episodes: 100%|███████████████████████████████████████████████████████████████████| 2000/2000 [00:02<00:00, 799.92it/s]


...

Estimated Value Function (excluding boundaries): [0.142  0.3273 0.5001 0.6788 0.8278]
RMS Error: 0.012819664446075157


Episodes: 100%|███████████████████████████████████████████████████████████████████| 2000/2000 [00:02<00:00, 853.12it/s]


Estimated Value Function (excluding boundaries): [0.1582 0.3405 0.4961 0.6843 0.8416]
RMS Error: 0.010186446142345174


Episodes: 100%|███████████████████████████████████████████████████████████████████| 2000/2000 [00:02<00:00, 868.16it/s]

Estimated Value Function (excluding boundaries): [0.1949 0.3497 0.5081 0.6438 0.802 ]
RMS Error: 0.022964020406212607


In [23]:
# Parameters
num_states = 7  # Total number of states
alpha = 0.01     # Learning rate
gamma = 1.0     # Discount factor
num_episodes = 10000

# Initialize state-value function
V = np.zeros(num_states)  # Initialize state-value function for all 7 states
V[1:6] = 0.5
# TD(0) algorithm
for episode in tqdm(range(num_episodes), desc="Episodes"):
    state = 3  # Start in the middle state
    while state != 0 and state != 6:  # Run until a terminal state is reached
        # Choose next state: move left or right
        next_state = state + np.random.choice([-1, 1])
        
        # Ensure next_state is within bounds (0 to 6)
        next_state = max(0, min(next_state, 6))
        
        # Assign reward based on whether next_state is terminal
        reward = 1 if next_state == 6 else 0
        
        # TD(0) update
        V[state] += alpha * (reward + gamma * V[next_state] - V[state])
        
        # Move to the next state
        state = next_state

# Print the estimated state-value function
print("Estimated State-Value Function:")
for state in range(num_states):
    print(f"V({state}) = {V[state]:.4f}")


Episodes: 100%|████████████████████████████████████████████████████████████████| 10000/10000 [00:07<00:00, 1379.97it/s]

Estimated State-Value Function:
V(0) = 0.0000
V(1) = 0.1447
V(2) = 0.3197
V(3) = 0.4952
V(4) = 0.6743
V(5) = 0.8454
V(6) = 0.0000
